# 00 · Runbook (Олимпиадный командный центр)

Этот ноутбук — панель управления всем пайплайном:
1) задаём параметры задачи,
2) быстро проверяем данные,
3) гоняем `tools/*` (features → model → blend → calib → submit),
4) следим за таймером и, если что, жмём **PANIC**.

Артефакты и состояние сессии: `artifacts/session/<SESSION_TAG>/state.json`  
Сабмиты: `artifacts/submissions/<RUN_TAG>/<profile>/`


In [ ]:
from IPython.display import display, HTML
display(HTML("""
<style>
/* ipywidgets v8 (JupyterLab 4) */
.jp-OutputArea .widget-button .widget-label { 
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
/* fallback для ipywidgets v7 */
.jupyter-widgets.widget-button .widget-label {
  white-space: normal !important; 
  overflow: visible !important; 
  text-overflow: clip !important;
  line-height: 1.2 !important;
}
</style>
"""))


In [ ]:
# %%capture
import os, sys, json, time, uuid, shlex, subprocess, textwrap, math, gc, pathlib, traceback
from pathlib import Path
from datetime import datetime
from typing import Dict, Any, List, Optional

import numpy as np
import pandas as pd

try:
    import ipywidgets as W
    from IPython.display import display, clear_output, HTML
except Exception as e:
    raise RuntimeError("Нужны ipywidgets: pip install ipywidgets") from e

ROOT = Path(".").resolve()
ART = ROOT / "artifacts"
SESSION_TAG = datetime.now().strftime("%Y%m%d_%H%M%S")
SESSION_DIR = ART / "session" / SESSION_TAG
SESSION_DIR.mkdir(parents=True, exist_ok=True)

STATE_PATH = SESSION_DIR / "state.json"
LOG_PATH = SESSION_DIR / "runbook.log"

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def log(msg: str):
    ts = datetime.now().strftime("%H:%M:%S")
    line = f"[{ts}] {msg}"
    print(line)
    with LOG_PATH.open("a", encoding="utf-8") as f:
        f.write(line + "\n")

STATE: Dict[str, Any] = {
    "SESSION_TAG": SESSION_TAG,
    "DATA_DIR": "data",
    "ID_COL": "id",
    "TARGET_COL": None,
    "TASK": "auto",  # auto|binary|multiclass|regression
    "SUBMIT_COL": "prediction",
    "RUN_TAG": f"run_{SESSION_TAG}",
    "SPLIT_KIND": "kfold",  # kfold|group|time
    "N_SPLITS": 5,
    "GROUP_COL": None,
    "DATE_COL": None,
    "TIME_EMBARGO": None,
    "SAFE": True,
    "FAST": True,
    "USE_CACHE": True,
    "GBDT_LIB": "lightgbm",
    "SEED": 42,
    "PRIMARY_METRIC": "roc_auc",
    "BEST_RUN_ID": None,
}

def save_state():
    STATE_PATH.write_text(json.dumps(STATE, ensure_ascii=False, indent=2), encoding="utf-8")
    log(f"state saved → {STATE_PATH}")

def load_state(path: Optional[Path] = None):
    p = path or STATE_PATH
    if not p.exists():
        log("state.json not found — skip load")
        return
    s = json.loads(p.read_text(encoding="utf-8"))
    STATE.update(s)
    log(f"state loaded ← {p}")

save_state()
log(f"SESSION_TAG={SESSION_TAG}")


In [ ]:
BTN_LAYOUT = W.Layout(min_width="220px", width="auto", height="36px", flex="0 0 auto")
ROW_LAYOUT = W.Layout(flex_flow="row wrap", grid_gap="8px")
GRID_LAYOUT = W.Layout(grid_template_columns="repeat(3, minmax(220px, 1fr))", grid_gap="8px")

In [ ]:
class CountDown:
    def __init__(self):
        self.budget_min = W.IntText(value=300, description="Бюджет (мин):", layout=W.Layout(width="260px"))
        self._t0 = None
        self._budget_s = 0
        self.bar = W.FloatProgress(value=0.0, min=0.0, max=1.0, description="Потрачено:", bar_style="")
        self.lbl = W.HTML(value="⏳ не запущен")
        self.btn_start = W.Button(description="Start timer", button_style="success", layout=BTN_LAYOUT)
        self.btn_stop = W.Button(description="Stop", button_style="", layout=BTN_LAYOUT)
        self.box = W.VBox([W.HBox([self.budget_min, self.btn_start, self.btn_stop], layout=ROW_LAYOUT), self.bar, self.lbl])
        self._running = False
        self.btn_start.on_click(self._on_start)
        self.btn_stop.on_click(self._on_stop)

    def _on_start(self, _):
        self._t0 = time.time()
        self._budget_s = max(60, int(self.budget_min.value) * 60)
        self._running = True
        self.lbl.value = f"▶️ таймер пошёл, бюджет {self._budget_s//60} мин"
        self._tick()

    def _on_stop(self, _):
        self._running = False
        self.lbl.value = "⏹ остановлен"

    def _tick(self):
        if not self._running:
            return
        elapsed = time.time() - self._t0
        left = max(0.0, self._budget_s - elapsed)
        self.bar.value = min(1.0, elapsed / self._budget_s)
        self.lbl.value = f"⌛ прошло: {int(elapsed//60)} мин {int(elapsed%60)} сек · осталось: {int(left//60)}:{int(left%60):02d}"
        # мягкий «ручной» тиканье
        from threading import Timer as _T
        _T(1.0, self._tick).start()

    def time_left_sec(self) -> int:
        if not self._t0:
            return int(self.budget_min.value)*60
        elapsed = time.time() - self._t0
        return max(0, int(self._budget_s - elapsed))

TIMER = CountDown()
display(HTML("<h3>Таймер тура</h3>")); display(TIMER.box)


In [ ]:
DATA_DIR_w = W.Text(value=STATE["DATA_DIR"], description="DATA_DIR:", layout=W.Layout(width="400px"))
ID_COL_w = W.Text(value=STATE["ID_COL"], description="ID_COL:", layout=W.Layout(width="250px"))
TARGET_COL_w = W.Text(value=str(STATE["TARGET_COL" ] or ""), description="TARGET_COL:", layout=W.Layout(width="300px"))
TASK_w = W.Dropdown(options=["auto","binary","multiclass","regression"], value=STATE["TASK"], description="TASK:")
SUBMIT_COL_w = W.Text(value=STATE["SUBMIT_COL"], description="SUBMIT_COL:")

RUN_TAG_w = W.Text(value=STATE["RUN_TAG"], description="RUN_TAG:", layout=W.Layout(width="400px"))
SPLIT_KIND_w = W.Dropdown(options=["kfold","group","time"], value=STATE["SPLIT_KIND"], description="SPLIT:")
N_SPLITS_w = W.IntText(value=STATE["N_SPLITS"], description="N_SPLITS:")
GROUP_COL_w = W.Text(value=str(STATE["GROUP_COL" ] or ""), description="GROUP_COL:")
DATE_COL_w = W.Text(value=str(STATE["DATE_COL" ] or ""), description="DATE_COL:")
TIME_EMBARGO_w = W.Text(value=str(STATE["TIME_EMBARGO" ] or ""), description="EMBARGO:")

SAFE_w = W.Checkbox(value=STATE["SAFE"], description="SAFE (anti-leak)")
FAST_w = W.Checkbox(value=STATE["FAST"], description="FAST")
USE_CACHE_w = W.Checkbox(value=STATE["USE_CACHE"], description="USE_CACHE")

GBDT_LIB_w = W.Dropdown(options=["lightgbm","xgboost","catboost"], value=STATE["GBDT_LIB"], description="GBDT_LIB:")
SEED_w = W.IntText(value=STATE["SEED"], description="SEED:")
PRIMARY_METRIC_w = W.Dropdown(options=["roc_auc","logloss","f1","rmse","r2"], value=STATE["PRIMARY_METRIC"], description="METRIC:")

btn_save_params = W.Button(description="Save params", button_style="primary", layout=BTN_LAYOUT)
out_params = W.Output()

def _save_params(_):
    STATE["DATA_DIR"] = DATA_DIR_w.value
    STATE["ID_COL"] = ID_COL_w.value
    STATE["TARGET_COL"] = TARGET_COL_w.value or None
    STATE["TASK"] = TASK_w.value
    STATE["SUBMIT_COL"] = SUBMIT_COL_w.value
    STATE["RUN_TAG"] = RUN_TAG_w.value

    STATE["SPLIT_KIND"] = SPLIT_KIND_w.value
    STATE["N_SPLITS"] = int(N_SPLITS_w.value)
    STATE["GROUP_COL"] = GROUP_COL_w.value or None
    STATE["DATE_COL"] = DATE_COL_w.value or None
    STATE["TIME_EMBARGO"] = TIME_EMBARGO_w.value or None

    STATE["SAFE"] = bool(SAFE_w.value)
    STATE["FAST"] = bool(FAST_w.value)
    STATE["USE_CACHE"] = bool(USE_CACHE_w.value)

    STATE["GBDT_LIB"] = GBDT_LIB_w.value
    STATE["SEED"] = int(SEED_w.value)
    STATE["PRIMARY_METRIC"] = PRIMARY_METRIC_w.value

    save_state()
    with out_params:
        clear_output()
        display(pd.DataFrame([STATE]))

btn_save_params.on_click(_save_params)

display(W.HTML("<h3>Параметры задачи</h3>"))
display(W.VBox([
    W.HBox([DATA_DIR_w, ID_COL_w, TARGET_COL_w], layout=ROW_LAYOUT),
    W.HBox([TASK_w, SUBMIT_COL_w, RUN_TAG_w], layout=ROW_LAYOUT),
    W.HBox([SPLIT_KIND_w, N_SPLITS_w, GROUP_COL_w, DATE_COL_w, TIME_EMBARGO_w], layout=ROW_LAYOUT),
    W.HBox([SAFE_w, FAST_w, USE_CACHE_w], layout=ROW_LAYOUT),
    W.HBox([GBDT_LIB_w, SEED_w, PRIMARY_METRIC_w, btn_save_params], layout=ROW_LAYOUT),
    out_params
]))


In [ ]:
btn_eda = W.Button(description="Run EDA-lite", button_style="", layout=BTN_LAYOUT)
out_eda = W.Output()

def run_eda(_):
    with out_eda:
        clear_output()
        try:
            data_dir = Path(STATE["DATA_DIR"])
            train = pd.read_csv(data_dir/"train.csv")
            test = pd.read_csv(data_dir/"test.csv")
            print("Train:", train.shape, "| Test:", test.shape)
            if STATE["TARGET_COL"] and STATE["TARGET_COL"] in train.columns:
                y = train[STATE["TARGET_COL"]]
                print("Target present. dtype:", y.dtype, "pos_rate:", float(y.mean()) if y.dtype.kind in "ifu" else "n/a")
            dupl = train.duplicated(subset=[STATE["ID_COL"]]).sum() if STATE["ID_COL"] in train.columns else 0
            print("ID duplicates in train:", dupl)
            const_cols = [c for c in train.columns if train[c].nunique(dropna=False)<=1]
            print("Constant cols:", const_cols[:10], "… total", len(const_cols))
            print("
Head(train):"); display(train.head(3))
            print("
Missing (train top-10):"); display(train.isna().mean().sort_values(ascending=False).head(10))
            eda = {
                "train_shape": list(train.shape),
                "test_shape": list(test.shape),
                "has_target": bool(STATE["TARGET_COL"] and STATE["TARGET_COL"] in train.columns),
                "const_cols": const_cols[:100]
            }
            (SESSION_DIR/"eda.json").write_text(json.dumps(eda, ensure_ascii=False, indent=2), encoding="utf-8")
            print("
Saved →", (SESSION_DIR/"eda.json"))
        except Exception as e:
            print("EDA error:", e)
            traceback.print_exc()

btn_eda.on_click(run_eda)
display(W.HTML("<h3>Sanity & EDA-lite</h3>"))
display(W.HBox([btn_eda], layout=ROW_LAYOUT), out_eda)


In [ ]:
btn_adv = W.Button(description="Run adversarial (quick)", button_style="", layout=BTN_LAYOUT)
out_adv = W.Output()

def _run_cmd(cmd: List[str], out: W.Output, cwd: Optional[Path]=None):
    with out:
        print("$", " \n".join(map(shlex.quote, cmd)))
        try:
            p = subprocess.Popen(cmd, cwd=str(cwd or ROOT), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
            for line in iter(p.stdout.readline, ''):
                print(line, end="")
            p.wait()
            print(f"
[exit {p.returncode}]")
        except Exception as e:
            print("Run error:", e)

def run_adversarial(_):
    out_adv.clear_output()
    tools = ROOT/"tools"/"adversarial.py"
    if not tools.exists():
        with out_adv: print("tools/adversarial.py не найден — пропускаю.")
        return
    args = [
        sys.executable, str(tools),
        "--data-dir", STATE["DATA_DIR"],
        "--id-col", STATE["ID_COL"],
        "--target-col", str(STATE["TARGET_COL"] or ""),
        "--quick"
    ]
    _run_cmd(args, out_adv)

btn_adv.on_click(run_adversarial)
display(W.HTML("<h3>Adversarial drift</h3>"))
display(btn_adv, out_adv)


In [ ]:
btn_build = W.Button(description="Build features", button_style="primary", layout=BTN_LAYOUT)
out_build = W.Output()

def build_features(_):
    out_build.clear_output()
    tools = ROOT/"tools"/"run_features.py"
    if not tools.exists():
        with out_build: print("tools/run_features.py не найден.")
        return
    args = [
        sys.executable, str(tools),
        "--data-dir", STATE["DATA_DIR"],
        "--id-col", STATE["ID_COL"],
        "--run-tag", STATE["RUN_TAG"],
        "--split-kind", STATE["SPLIT_KIND"],
        "--n-splits", str(STATE["N_SPLITS"]),
        "--seed", str(STATE["SEED"]),
    ]
    if STATE["TARGET_COL"]:
        args += ["--target-col", STATE["TARGET_COL"]]
    if STATE["GROUP_COL"]:
        args += ["--group-col", STATE["GROUP_COL"]]
    if STATE["DATE_COL"]:
        args += ["--date-col", STATE["DATE_COL"]]
    if STATE["TIME_EMBARGO"]:
        args += ["--time-embargo", STATE["TIME_EMBARGO"]]
    if STATE["SAFE"]:
        args += ["--safe"]
    if STATE["FAST"]:
        args += ["--fast"]
    if STATE["USE_CACHE"]:
        args += ["--use-cache"]
    _run_cmd(args, out_build)
    # показать паспорт сета, если есть
    try:
        meta = json.loads((ROOT/"artifacts"/"sets"/STATE["RUN_TAG"]/ "meta.json").read_text(encoding="utf-8"))
        with out_build:
            print("
— Catalog:"); display(pd.json_normalize(meta).T)
    except Exception:
        pass

btn_build.on_click(build_features)
display(W.HTML("<h3>Сбор фич</h3>"))
display(btn_build, out_build)


In [ ]:
btn_valset = W.Button(description="Validate set", button_style="", layout=BTN_LAYOUT)
out_valset = W.Output()

def validate_set(_):
    out_valset.clear_output()
    tools = ROOT/"tools"/"validate_set.py"
    if not tools.exists():
        with out_valset: print("tools/validate_set.py не найден.")
        return
    args = [
        sys.executable, str(tools),
        "--tag", STATE["RUN_TAG"]
    ]
    _run_cmd(args, out_valset)

btn_valset.on_click(validate_set)
display(W.HTML("<h3>Валидация сета</h3>"))
display(btn_valset, out_valset)


In [ ]:
USE_GBDT_w = W.Checkbox(value=True, description="GBDT on DENSE")
USE_LIN_w = W.Checkbox(value=True, description="Linear on SPARSE")
USE_HYB_w = W.Checkbox(value=False, description="Hybrid extra")
btn_train = W.Button(description="Train models", button_style="warning", layout=BTN_LAYOUT)
out_train = W.Output()

def train_models(_):
    out_train.clear_output()
    tools = ROOT/"tools"/"run_model.py"
    if not tools.exists():
        with out_train: print("tools/run_model.py не найден.")
        return
    args = [
        sys.executable, str(tools),
        "--tag", STATE["RUN_TAG"],
        "--gbdt-lib", STATE["GBDT_LIB"],
        "--n-splits", str(STATE["N_SPLITS"]),
        "--seed", str(STATE["SEED"]),
        "--primary-metric", STATE["PRIMARY_METRIC"]
    ]
    if USE_GBDT_w.value: args += ["--use-gbdt"]
    if USE_LIN_w.value: args += ["--use-linear"]
    if USE_HYB_w.value: args += ["--use-hybrid"]
    _run_cmd(args, out_train)

    # Подтянуть свежие модели и дать выбрать BEST
    try:
        mdir = ROOT/"artifacts"/"models"
        rows = []
        for d in sorted(mdir.glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)[:100]:
            met = d/"metrics.json"
            if met.exists():
                js = json.loads(met.read_text(encoding="utf-8"))
                rows.append({"run_id": d.name, "cv_mean": js.get("cv_mean"), "cv_std": js.get("cv_std"), "lib": js.get("lib")})
        if rows:
            df = pd.DataFrame(rows).sort_values("cv_mean", ascending=False)
            with out_train:
                print("
Fresh models:"); display(df.head(20))
            # установить лучшую по таблице
            STATE["BEST_RUN_ID"] = str(df.iloc[0]["run_id"])
            save_state()
            with out_train:
                print("BEST_RUN_ID →", STATE["BEST_RUN_ID"])
    except Exception:
        pass

display(W.HTML("<h3>Обучение моделей</h3>"))
display(W.HBox([USE_GBDT_w, USE_LIN_w, USE_HYB_w, btn_train], layout=ROW_LAYOUT), out_train)
btn_train.on_click(train_models)


In [ ]:
btn_refresh_runs = W.Button(description="Обновить список моделей", button_style="", layout=BTN_LAYOUT)
runs_select = W.SelectMultiple(options=[], description="runs", rows=8, layout=W.Layout(width="600px"))
btn_blend_eq = W.Button(description="Blend: equal-weight", button_style="info", layout=BTN_LAYOUT)
btn_blend_l2 = W.Button(description="Blend: level-2 (ridge)", button_style="", layout=BTN_LAYOUT)
out_blend = W.Output()

def refresh_runs(_=None):
    mdir = ROOT/"artifacts"/"models"
    options = []
    for d in sorted(mdir.glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)[:200]:
        met = d/"metrics.json"
        if met.exists():
            js = json.loads(met.read_text(encoding="utf-8"))
            options.append((f"{d.name} | cv={js.get('cv_mean')}", d.name))
    runs_select.options = options

btn_refresh_runs.on_click(refresh_runs)
refresh_runs()

def blend_eq(_):
    out_blend.clear_output()
    tools = ROOT/"tools"/"run_blend.py"
    if not tools.exists():
        with out_blend: print("tools/run_blend.py не найден.")
        return
    chosen = list(runs_select.value)
    if len(chosen) < 2:
        with out_blend: print("Нужно ≥2 run_id для бленда.")
        return
    args = [sys.executable, str(tools), "--runs", *chosen, "--equal"]
    _run_cmd(args, out_blend)

def blend_l2(_):
    out_blend.clear_output()
    tools = ROOT/"tools"/"run_blend.py"
    if not tools.exists():
        with out_blend: print("tools/run_blend.py не найден.")
        return
    chosen = list(runs_select.value)
    if len(chosen) < 2:
        with out_blend: print("Нужно ≥2 run_id для бленда.")
        return
    args = [sys.executable, str(tools), "--runs", *chosen, "--level2", "ridge"]
    _run_cmd(args, out_blend)

btn_blend_eq.on_click(blend_eq)
btn_blend_l2.on_click(blend_l2)

display(W.HTML("<h3>BlendLab</h3>"))
display(W.HBox([btn_refresh_runs], layout=ROW_LAYOUT), runs_select, W.HBox([btn_blend_eq, btn_blend_l2], layout=ROW_LAYOUT), out_blend)


In [ ]:
best_input = W.Text(value=str(STATE["BEST_RUN_ID" ] or ""), description="BEST_RUN_ID:", layout=W.Layout(width="520px"))
btn_set_best = W.Button(description="Set as BEST", button_style="", layout=BTN_LAYOUT)
btn_calib = W.Button(description="Calibrate (Platt) + τ", button_style="info", layout=BTN_LAYOUT)
out_calib = W.Output()

def set_best(_):
    STATE["BEST_RUN_ID"] = best_input.value.strip() or None
    save_state()
    with out_calib:
        print("BEST_RUN_ID saved:", STATE["BEST_RUN_ID"])

def platt_and_tau(_):
    out_calib.clear_output()
    if not STATE["BEST_RUN_ID"]:
        with out_calib: print("BEST_RUN_ID не задан.")
        return
    rdir = ROOT/"artifacts"/"models"/STATE["BEST_RUN_ID"]
    oof_true = rdir/"oof_true.npy"
    oof_pred = rdir/"oof_pred.npy"
    test_pred = rdir/"test_pred.npy"
    if not (oof_true.exists() and oof_pred.exists()):
        with out_calib: print("OOF файлы не найдены — возможно, это бленд без OOF. Можно калибровать исходные.")
        return
    y = np.load(oof_true)
    p = np.load(oof_pred)
    # Platt через логрег
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression(C=1.0, max_iter=200)
    lr.fit(p.reshape(-1,1), y.astype(int))
    a = float(lr.coef_.ravel()[0]); b = float(lr.intercept_.ravel()[0])
    def _sig(z): return 1.0/(1.0+np.exp(-z))
    p_cal = _sig(a*p + b)
    # поиск τ по F1
    grid = np.linspace(0.05, 0.95, 37)
    best_f1, best_t = -1.0, 0.5
    from sklearn.metrics import f1_score, roc_auc_score
    for t in grid:
        f1 = f1_score(y, (p_cal>=t).astype(int))
        if f1>best_f1:
            best_f1, best_t = f1, float(t)
    # сохранить
    (rdir/"calibrator.json").write_text(json.dumps({"a":a,"b":b}, ensure_ascii=False, indent=2), encoding="utf-8")
    (rdir/"thresholds.json").write_text(json.dumps({"tau":best_t}, ensure_ascii=False, indent=2), encoding="utf-8")
    with out_calib:
        print("Platt: a=%.4f b=%.4f | AUC=%.5f | best τ=%.3f (F1=%.4f)" % (a,b, roc_auc_score(y,p_cal), best_t, best_f1))
        if test_pred.exists():
            tp = np.load(test_pred)
            tp_cal = _sig(a*tp + b)
            np.save(rdir/"test_pred_calibrated.npy", tp_cal)
            print("Saved test_pred_calibrated.npy")

btn_set_best.on_click(set_best)
btn_calib.on_click(platt_and_tau)

display(W.HTML("<h3>Калибровка & τ</h3>"))
display(W.HBox([best_input, btn_set_best, btn_calib], layout=ROW_LAYOUT), out_calib)


In [ ]:
btn_stab = W.Button(description="Run stability for BEST", button_style="", layout=BTN_LAYOUT)
out_stab = W.Output()

def run_stability(_):
    out_stab.clear_output()
    if not STATE["BEST_RUN_ID"]:
        with out_stab: print("BEST_RUN_ID не задан.")
        return
    tools = ROOT/"tools"/"stability.py"
    if not tools.exists():
        with out_stab: print("tools/stability.py не найден.")
        return
    args = [sys.executable, str(tools), "--run", STATE["BEST_RUN_ID"], "--quick"]
    _run_cmd(args, out_stab)

btn_stab.on_click(run_stability)
display(W.HTML("<h3>Stability</h3>"))
display(btn_stab, out_stab)


In [ ]:
SUBMIT_DIR_w = W.Text(value=str((ROOT/"artifacts"/"submissions").as_posix()), description="SUB_DIR:", layout=W.Layout(width="520px"))
btn_submit = W.Button(description="Make submission (BEST)", button_style="success", layout=BTN_LAYOUT)
out_submit = W.Output()

def make_submit(_):
    out_submit.clear_output()
    if not STATE["BEST_RUN_ID"]:
        with out_submit: print("BEST_RUN_ID не задан.")
        return
    tools = ROOT/"tools"/"submit.py"
    if not tools.exists():
        with out_submit: print("tools/submit.py не найден.")
        return
    args = [
        sys.executable, str(tools),
        "--best", STATE["BEST_RUN_ID"],
        "--id-col", STATE["ID_COL"],
        "--submit-col", STATE["SUBMIT_COL"],
        "--run-tag", STATE["RUN_TAG"],
        "--out-dir", SUBMIT_DIR_w.value
    ]
    _run_cmd(args, out_submit)

btn_submit.on_click(make_submit)
display(W.HTML("<h3>Сабмит</h3>"))
display(W.HBox([SUBMIT_DIR_w, btn_submit], layout=ROW_LAYOUT), out_submit)


In [ ]:
NO_TEXT_w = W.Checkbox(value=False, description="--no-text")
NO_CALIB_w = W.Checkbox(value=False, description="--no-calibration")
btn_panic = W.Button(description="PANIC: use time left", button_style="danger", layout=BTN_LAYOUT)
out_panic = W.Output()

def run_panic(_):
    out_panic.clear_output()
    tools = ROOT/"tools"/"panic.py"
    if not tools.exists():
        with out_panic: print("tools/panic.py не найден.")
        return
    time_left = max(1, TIMER.time_left_sec()//60)
    args = [
        sys.executable, str(tools),
        "--time-budget-min", str(time_left),
        "--data-dir", STATE["DATA_DIR"],
        "--id-col", STATE["ID_COL"],
        "--name", "panic",
        "--gbdt-lib", STATE["GBDT_LIB"]
    ]
    if STATE["TARGET_COL"]:
        args += ["--target-col", STATE["TARGET_COL"], "--task", STATE["TASK"] if STATE["TASK"]!="auto" else "binary"]
    if STATE["RUN_TAG"]:
        args += ["--tag", STATE["RUN_TAG"]]
    if NO_TEXT_w.value: args.append("--no-text")
    if NO_CALIB_w.value: args.append("--no-calibration")
    _run_cmd(args, out_panic)

btn_panic.on_click(run_panic)
display(W.HTML("<h3>Аварийный режим</h3>"))
display(W.HBox([NO_TEXT_w, NO_CALIB_w, btn_panic], layout=ROW_LAYOUT), out_panic)


In [ ]:
TTL_days_w = W.IntText(value=7, description="TTL days:")
KEEP_last_w = W.IntText(value=2, description="Keep last per block:")
DRY_w = W.Checkbox(value=True, description="dry-run")
btn_gc = W.Button(description="GC artifacts", button_style="", layout=BTN_LAYOUT)
out_gc = W.Output()

def run_gc(_):
    out_gc.clear_output()
    tools = ROOT/"tools"/"gc_artifacts.py"
    if not tools.exists():
        with out_gc: print("tools/gc_artifacts.py не найден.")
        return
    args = [
        sys.executable, str(tools),
        "--ttl-days", str(int(TTL_days_w.value)),
        "--keep-last-per-block", str(int(KEEP_last_w.value))
    ]
    if DRY_w.value: args.append("--dry-run")
    _run_cmd(args, out_gc)

btn_gc.on_click(run_gc)
display(W.HTML("<h3>GC артефактов</h3>"))
display(W.HBox([TTL_days_w, KEEP_last_w, DRY_w, btn_gc], layout=ROW_LAYOUT), out_gc)


In [ ]:
btn_save = W.Button(description="Save state", button_style="primary", layout=BTN_LAYOUT)
btn_load = W.Button(description="Load state", button_style="", layout=BTN_LAYOUT)
out_state = W.Output()

def _do_save(_):
    save_state()
    with out_state: 
        clear_output(); print("Saved", STATE_PATH)

def _do_load(_):
    load_state()
    with out_state:
        clear_output(); display(pd.DataFrame([STATE]))

btn_save.on_click(_do_save)
btn_load.on_click(_do_load)
display(W.HTML("<h3>State</h3>"))
display(W.HBox([btn_save, btn_load], layout=ROW_LAYOUT), out_state)


### Чек-лист перед стартом
- [ ] TARGET верный / без утечек
- [ ] SPLIT соответствует условию (kfold/group/time)
- [ ] Бюджет времени задан, таймер запущен
- [ ] Быстрый adversarial не сигналит «красным»
- [ ] RUN_TAG сохранён, сет провалидирован
- [ ] BEST_RUN_ID задан перед сабмитом


* Все кнопки пишут лог в `artifacts/session/<SESSION_TAG>/runbook.log`.
* Если какой-то `tools/*` отсутствует — клетка просто подскажет и не упадёт.
* Для калибровки и τ есть локальный fallback (Platt/логрег + F1-грид).
* PANIC использует `time_left` с таймера и ваш `RUN_TAG` (если уже собран сет).
